In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Load both left and right eye dataset
base_path = 'C:/Users/manik/DL-Project/DeepLearning/DL-Project'
left_df = pd.read_excel(f'{base_path}/updated_left_eye.xlsx')
right_df = pd.read_excel(f'{base_path}/updated_right_eye.xlsx')


In [3]:
left_df.head()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_left.jpg,cataract,0,0,0,1,0,0,0,0
1,1,1_left.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_left.jpg,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,3_left.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4,4_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1


In [4]:
right_df.head()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_right.jpg,normal fundus,1,0,0,0,0,0,0,0
1,1,1_right.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3,3,3_right.jpg,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,4_right.jpg,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0


In [5]:
df = pd.concat([left_df, right_df], axis=0)
df.head()


,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_left.jpg,cataract,0,0,0,1,0,0,0,0
1,1,1_left.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_left.jpg,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,3_left.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4,4_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1


In [6]:
df.tail()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
3478,3495,4686_right.jpg,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
3479,3496,4688_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3480,3497,4689_right.jpg,normal fundus,1,0,0,0,0,0,0,0
3481,3498,4690_right.jpg,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0
3482,3499,4784_right.jpg,hypertensive retinopathy，age-related macular d...,0,0,0,0,1,1,0,0


In [8]:
df.shape


(6968, 11)

In [9]:
# Input pipeline
image_name = df['Fundus']
image_name.head()

0    0_left.jpg
1    1_left.jpg
2    2_left.jpg
3    3_left.jpg
4    4_left.jpg
Name: Fundus, dtype: object

In [14]:
labels = df.oc[:]
labels.head()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
3,3,3_left.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4,4_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1
5,5,5_left.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
6,6,6_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1
7,7,7_left.jpg,drusen,0,0,0,0,0,0,0,1
